[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adan-rs/amd/blob/main/notebooks/24_Analisis_factorial.ipynb)

# Análisis factorial exploratorio

El análisis factorial es una técnica de reducción de dimensiones. Esto es, se busca “reducir” una gran cantidad de variables a un conjunto más pequeño de factores o componentes. Esta técnica se utiliza para comprender la estructura subyacente de un conjunto de variables, construir una escala que mida una variable no observable (variable latente), o reducir un conjunto de variables a un tamaño más manejable.


El análisis factorial puede ser exploratorio o confirmatorio. El análisis factorial exploratorio se utiliza comúnmente en etapas iniciales de la investigación para analizar la interrelación entre las variables. El análisis factorial confirmatorio es una técnica en la que se busca confirmar hipótesis específicas acerca de la estructura subyacente en un conjunto de variables. 

In [ ]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

Instalar solamente una vez: `!pip install factor_analyzer`

El archivo “Base 05 Jóvenes” contiene los resultados de una encuesta aplicada por estudiantes de una clase de estadística en la FSEV UK (Eslovaquia) en 2013. El archivo consta de las respuestas de 1010 jóvenes entre 15 y 30 años. Los temas de la encuesta son: preferencias musicales (19 ítems), preferencias por tipo de películas (12 ítems), uso de internet, hábitos de gasto (7 ítems), y datos demográficos (10 ítems). Se realizará un análisis factorial exploratorio con las variables de preferencias por películas.

In [ ]:
df = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/jovenes.xlsx')
df.columns

In [ ]:
movies = ['Horror', 'Thriller', 'Comedy', 'Romantic', 'Scifi', 'War', 
         'FantasyFairytales', 'Animated', 'Documentary', 'Western']
df = df[movies]

In [ ]:
df.info()

## Preparación de datos

In [ ]:
# Identificar filas con valores faltantes
filas_perdidos = df.isna().any(axis=1).sum()
print(f"Número de filas con valores perdidos: {filas_perdidos}")

In [ ]:
# Eliminar filas con valores faltantes
df = df.dropna()

In [ ]:
# Estandarizar las variables
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

**Tamaño muestral**: Existen varias recomendaciones en cuanto al tamaño muestral. Tabachnick y Fidell (2001) recomiendan tener al menos 300 casos, pero aceptan que muestras más pequeñas son suficientes bajo ciertas condiciones. Nunnally (1978) recomienda tener al menos 10 observaciones por cada variable.

**Tipo de variables**: Aunque es preferible que las variables sean cuantitativas (de escala), en la práctica se pueden utilizar variables con escala de medición ordinal de cinco o más niveles. Si es así, se requiere que la distancia entre cada categoría sea igual (es decir, que los intervalos sean regulares). Se asume que las escalas tipo Likert cumplen con esta condición.

**Condiciones de aplicabilidad**: El procedimiento más común para realizar el análisis de factores se basa en el análisis de componentes principales. En el análisis de componentes principales se utilizan las correlaciones entre conjuntos de ítems (variables) para calcular los factores que los representan. Por esta razón, como condición de aplicabilidad, es deseable obtener la matriz de correlaciones y asegurarse que varios coeficientes sean mayores que 0.3. 

Sin embargo, para evaluar la conveniencia del análisis factorial, es preferible revisar la prueba de esfericidad de Bartlett y el estadístico Kaiser-Meyer-Olkin (KMO). La prueba de esfericidad de Bartlett prueba la hipótesis nula de que matriz de correlación es una matriz diagonal (todos los elementos son cero excepto la diagonal principal). Debido a que se espera que existan varias correlaciones significativas, lo deseable es rechazar la hipótesis nula (p < 0.05). 

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

print('Chi cuadrado', calculate_bartlett_sphericity(df)[0])
print('P valor', calculate_bartlett_sphericity(df)[1])

El estadístico KMO indica si las correlaciones entre variables pueden ser explicadas por las otras variables. Es deseable obtener valores del KMO de al menos 0.50.

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo

kmo_all, kmo_model = calculate_kmo(df[df.columns])
print('KMO score:',kmo_model)

*Interpretación*: Se considera que el análisis factorial es adecuado dado que se obtuvo un valor mayor a 0.5 en el indicador KMO y la significancia de la prueba de esfericidad de Bartlett es menor que 0.05.

## Extracción de factores
Existen varios métodos de extracción de factores. La técnica más utilizada es la de análisis de componentes principales. Otras opciones máxima verosimilitud ('ml') o bien el método predeterminado "minres" que es más robusto ante violaciones de normalidad.

Se recomienda que si el objetivo es reducir las variables se utilice componentes principales, y si el objetivo es identificar dimensiones latentes se utilice la factorización de ejes principales. Sin embargo, aunque difieren en lo teórico, los resultados suelen ser similares por lo que generalmente se puede utilizar análisis de componentes principales.

In [ ]:
from factor_analyzer import FactorAnalyzer

fa = FactorAnalyzer(rotation='varimax', method='principal')
fa.fit(df)

**Rotación de factores**: Existen dos opciones para la rotación de factores, ortogonal u oblicua. La rotación ortogonal lleva a una solución que es más fácil de interpretar, sin embargo, requiere asumir que los constructos subyacentes son independientes. La rotación oblicua, en cambio, permite asumir que los factores están correlacionados.

Dentro de las técnicas de rotación ortogonal, la más utilizada es el método varimax. La técnica más común en rotación oblicua es oblimin directo, y requiere establecer un valor delta que determina el nivel de correlación entre factores. El valor predeterminado de cero implica que los factores están moderadamente correlacionados

Si los patrones de los factores son claros, ambos tipos de técnicas llevarán a resultados similares. Se puede partir de una rotación varimax. Si los resultados son difíciles de interpretar se recomienda una rotación oblicua. 

## Determinación del número de factores
El investigador debe determinar el número de factores que se consideran deseables. Algunas técnicas que pueden ayudar en esta decisión son el criterio de Kaiser o el gráfico de sedimentación (scree plot). Bajo el criterio de Kaiser (que es el más empleado) solamente los factores con un autovalor de 1.0 o más deben ser considerados. Los autovalores (eigenvalues) es una métrica de la cantidad de varianza explicada por cada factor. Un autovalor de 1.0 representa la varianza de una única variable.

El gráfico de sedimentación (scree plot) implica graficar cada uno de los valores propio del factor y revisar la gráfica para ver el punto en el cual la curva cambia de dirección y se vuelve horizontal. Se recomienda retener todos los factores por encima de ese punto de quiebre (es decir, uno menos que el punto de quiebre). Si el gráfico de sedimentación no es claro, se puede tomar la decisión con base en el criterio de Kaiser.

In [ ]:
# Cálculo de autovalores
ev, v = fa.get_eigenvalues()

# Scree plot
plt.scatter(range(1, 10), ev[:9])
plt.plot(range(1,10), ev[:9])
plt.title('ScreePlot');

In [ ]:
k = 3
fa = FactorAnalyzer(rotation = 'varimax', n_factors=k, method='principal')
fa.fit(df)

La comunalidad indica qué proporción de la varianza de una variable es varianza común. Aunque no existe un mínimo, las variables con una comunalidad baja (menores a 0.30) son candidatas a ser descartadas en el análisis. 

In [ ]:
communalities = pd.DataFrame(fa.get_communalities(), index=list(df.columns))
communalities.rename(columns={0: 'Comunalidades'}, inplace=True)
communalities.sort_values(by='Comunalidades')

**Cargas factoriales**:  Una vez extraídos los factores, la correlación entre una variable y el factor se llama carga factorial. Si son pocos factores, las cargas factoriales deberán ser de al menos 0.50, pero si son muchos factores se pueden considerar arriba de 0.30

In [ ]:
fa_loadings = pd.DataFrame(fa.loadings_,
                           columns=['Factor{}'.format(i) for i in range(1, k+1)],
                           index=df.columns)
fa_loadings['Factor_asignado'] = fa_loadings.abs().idxmax(axis=1)
fa_loadings

Las cargas más altas suelen ayudar a identificar la variable latente en cada componente. En la presencia de cargas cruzadas (variables con cargas importantes en más de un componente) se recomienda volver a realizar el análisis cambiando el número de componentes, utilizando otra técnica de rotación, o en el caso extremo, eliminar esa variable. 

Una vez identificados los factores, se debe crear una etiqueta que describa cada factor. Considere que el análisis factorial exploratorio es una técnica de exploración de datos, por tanto, la interpretación queda al juicio del investigador. 

In [ ]:
factors = list(fa_loadings.columns[:-1])
# Modificando formato
table = fa_loadings.sort_values(by='Factor_asignado').style.map(
    lambda x: 'background-color: yellow' if abs(x) > 0.4 else '',
    subset=factors)
table

**Presentación de resultados**:  
>"Se realizó un análisis factorial exploratorio con los 11 ítems de preferencias por películas mediante el método de análisis de componentes principales con rotación varimax. La medida Kaiser-Meyer-Olkin de adecuación de muestreo, KMO = 0.637, es superior al límite aceptable de 0.50. El resultado de la prueba de esfericidad de Bartlett, $\chi^2$ = 1745.17, p < 0.001, respalda también el empleo del análisis factorial. En el análisis inicial se obtuvieron tres componentes con autovalores mayores a 1. El primer componente se relaciona con películas "Aventuras", el segundo componente son películas "Fantasía", y el tercer componente son películas "Inquietantes". La tabla muestra las cargas factoriales después de la rotación."

**Documentación**
https://factor-analyzer.readthedocs.io/en/latest/factor_analyzer.html

## Preguntas de autoevaluación

**Pregunta 1**. Una empresa de investigación de mercados está analizando un conjunto de datos de encuestas para identificar las dimensiones que influyen en la satisfacción del cliente. Antes de proceder con un análisis factorial, el equipo decide verificar si las variables están suficientemente correlacionadas para justificar este análisis. ¿Qué prueba estadística debe utilizar el equipo para evaluar si es apropiado realizar un análisis factorial?

A) Prueba de esfericidad de Bartlett

B) Prueba de independencia Chi cuadrado

B) Prueba de homocedasticidad de Levene

**Pregunta 2**. Una empresa financiera está analizando grandes volúmenes de datos para tomar decisiones estratégicas. A continuación se describen tres escenarios. ¿En cuál de ellos sería más útil aplicar un análisis factorial?

A) El equipo de finanzas quiere prever los flujos de caja futuros utilizando variables como ingresos, gastos, e inversiones pasadas, y necesita un modelo que prediga directamente estos valores.

B) El equipo de gestión de riesgos necesita reducir la cantidad de variables económicas para identificar unos pocos factores clave que explican la mayor parte de la variabilidad en los índices de mercado y así poder modelar la relación entre estos factores y el riesgo de la cartera.

C) El departamento de auditoría está revisando las transacciones financieras individuales para detectar fraudes, utilizando reglas de negocio específicas para identificar anomalías en las transacciones.